In [59]:
import abc
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
import numpy as np
from glob import glob
from natsort import natsorted

In [78]:
@dataclass
class FE_contrib_init:
    kB_kcalmolK: float = 0.001987204259
    TemperatureK: np.ndarray = np.array(300)
    betaFactor: np.ndarray = 1 / (kB_kcalmolK*TemperatureK)
    
    def getCVs(self, file_parser: FileParser) -> None:
        file_parser.ParseCollectiveVariables
    

In [81]:

@dataclass
class FileParser(ABC):
    files: list[str] = field(init=False)
    Samplesizes: np.ndarray = field(init=False)
    OriginH_CollectiveVariables: list[np.ndarray] = field(init=False)
    
    @abstractmethod
    def get_anchor_and_forceConstant(self, anchorRegex: str, forceConstantRegex: str) -> np.ndarray:
        '''abstract method to obtain force constant and anchor information form simulation output files'''
    
    @abstractmethod
    def ParseCollectiveVariables(self, files) -> list[np.ndarray]:
        '''abstract method to obtain raw data from simulation output''' 
    
    def calculateSamplesize(self) -> list[int]:
        pass
    

class PMD_FileParser(FileParser):
    pass
    def get_anchor_and_forceConstant(self, anchorRegex: str, forceConstantRegex: str) -> np.ndarray:
        print('gettinganchor and constant')
    
    def ParseCollectiveVariables(self, files) -> list[np.ndarray]:
        print('getting collective variables from simulation data') 
    
def main():
    files = natsorted(glob('../_sim/13*/_output/pmd_p*[!a].txt'))
    dG_oS_init = FE_contrib_init()
    dG_oS_init.getCVs(PMD_FileParser)
    # dG_oS_dataloader.decorrelate(decorrelationScheme())
    # dG_oS_Energy = FE_contribution('PMF-like')
    
if __name__ == '__main__':
    main()

<class '__main__.PMD_FileParser'>
